## 참조모델 단어 추출
- 영어, 한글 용어 따로 단어 추출

In [14]:
#라이브러리 import

import pandas as pd
from openpyxl import load_workbook
import os
import re
from collections import Counter

dir_path = os.getcwd()
dir_path = os.path.join(dir_path, "file")
print(dir_path)

c:\Users\PC\Desktop\git_repo\ML-DL-Training\Data_preprocess\workspace\file


### 1차 데이터 처리 작업

- 파일 로딩
- 토큰화를 통한 단어 분리
- 특수 문자 제거 및 숫자 제거
- 불용어 제거, 길이가 2이하인 단어 제거

In [15]:
def excel_file_load(dir_path, file_name, sheet_name):
    file_path = os.path.join(dir_path,file_name)
    file = pd.read_excel(file_path, sheet_name = sheet_name)
    return file

In [16]:
refer_file = excel_file_load(dir_path, "참조모델데이터.xlsx", "영어 용어")
refer_file = refer_file.dropna()
refer_file

,Data
0,AbnormalityDetection
1,Access
2,Algorithm
3,Approval
4,AttenuationRate
...,...
125,WarrantyDate
126,WorkforceSize
127,WorkinprocessReductionRate
128,ZeroEmission


In [17]:
#nltk 라이브러리 import

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

#불용어
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


- 각 기능별 함수 정의

In [29]:

# 품사태그를 WordNet의 태그로 변환
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return None

# Definition 문장 전처리 함수
def process_sentence(text):
    # 특수문자를 공백으로 대체
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    # 연속된 공백을 하나로 축소
    text = re.sub(r'\s+', ' ', text).strip()
    # 공백 기준으로 단어 분리
    words = text.split()
    # 각 단어를 대문자 기준으로 다시 분리 (리스트 컴프리헨션 사용)
    split_words = [subword for word in words for subword in re.findall(r'[A-Z][a-z]*', word)]
    # 품사 태깅
    tagged_tokens = pos_tag(split_words)
    # 불용어 리스트 가져오기
    stop_words = set(stopwords.words('english'))
    # Lemmatizer 객체 생성
    lemmatizer = WordNetLemmatizer()
    # 불용어 제거 및 단어의 원형 추출
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag) or nltk.corpus.wordnet.NOUN)
        for word, tag in tagged_tokens if word.lower() not in stop_words and len(word) > 2
    ]
    return lemmatized_tokens

In [30]:
#각 column 데이터에 따른 데이터프레임 생성
def word_df_create(df, column_name):

    # 해당 열의 단어 추출 및 처리
    all_words = []
    for sentence in df[column_name].dropna():
        words = process_sentence(sentence)
        all_words.extend(words)

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(set(all_words), columns=['Word'])

    return result_df    

In [31]:
#column_name_1: Preferred name -> pre_df 데이터 프레임 생성
#column_name_2: Definition -> def_df 데이터 프레임 생성

pre_df = word_df_create(refer_file, 'Data')


In [45]:

df_lower = pre_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df_lower

C:\Users\PC\AppData\Local\Temp\ipykernel_15020\3226740574.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lower = pre_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,Word
0,dangerous
1,postprocessing
2,emission
3,time
4,revenue
...,...
141,technical
142,diagnostic
143,contaminant
144,return


In [46]:
# 결과 엑셀 시트에 분할 저장

result_file_path = os.path.join(dir_path,'참조모델 데이터 저장.xlsx')
with pd.ExcelWriter(result_file_path) as writer:
    df_lower.to_excel(writer, sheet_name='Sheet1', index=False)

### 2차 데이터 처리 작업

- 대문자 중복 단어 제거
- 축약어, 단어 분리 처리
- 단어 표제어 변환
- 각 column 별 데이터 추출 후 병합, 중복 처리
- 최종 데이터 엑셀파일 저장   
 **-> sheet1: 단어 데이터, sheet2: 대문자 축약어 데이터**

- 표제어 변환

In [35]:
from textblob import TextBlob, Word
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet, words

nltk.download('omw-1.4')
nltk.download('words')

EN_words = set(words.words())

import inflect

p = inflect.engine()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [36]:
# Steemmer를 사용하여 단어의 원형 추출
def stemmer(word):
    ps = PorterStemmer()
    return ps.stem(word)

# 단수형 변환 함수 정의
def to_singular(word):
    singular_word = p.singular_noun(word)
    return singular_word if singular_word else word

# 영어 단어 인지 확인
def is_EN_word(word):
    if word in EN_words:
        return word
    
lemmatizer = WordNetLemmatizer()

# 원형을 추출
def get_lemma(word):
    pos = pos_tag([word])[0][1]  # 단어의 품사 태깅
    wordnet_pos = get_wordnet_pos(pos)  # WordNet의 품사 태그로 변환
    lemma = lemmatizer.lemmatize(word, wordnet_pos)  # 원형 추출
    return lemma

# 2이하인 값 제거
def remove_short_words(cell):
    return cell if len(cell) > 2 else None

In [37]:
# 표제어 추출
def Lemmatization_process(word_data):
    # 데이터프레임에 단어의 원형 추출
    word_result = pd.DataFrame()

    # 추출 단어의 단수형 추출, nltk 내의 word에서 존재여부 파악, 중복 처리
    word_result['Word'] = word_data['Word'].apply(lambda x: is_EN_word(to_singular(x))).dropna().drop_duplicates()

    # 추출 단어의 어간 추출(Stemming)
    word_result['Stemm_Word'] = word_result['Word'].apply(stemmer)

    # 어간 추출 단어와 기존 단어의 비교
    word_result['Data_Equal'] = word_result.eval('Word == Stemm_Word')

    # 어간 추출 단어와 다른 단어중 nltk 내의 word에서 존재여부 파악 
    Stemm_Word = word_result[word_result['Data_Equal'] == False][['Stemm_Word']]
    word_result['Stem_con'] = Stemm_Word['Stemm_Word'].apply(is_EN_word)

    # 최종 결과 col생성
    word_result['Result_word'] = word_result.apply(
        lambda row: row['Word'] if row['Data_Equal'] else (row['Stemm_Word'] if row['Stem_con'] else row['Word']),axis=1
    )

    # 2이하인 값 제거
    word_result['Result_word'] = word_result['Result_word'].apply(remove_short_words)

    return word_result['Result_word'].drop_duplicates().to_frame(name='Word').reset_index(drop=True)

In [47]:
# 최종 단어 추출
pre_result_word = Lemmatization_process(df_lower).dropna()

In [48]:
pre_result_word

,Word
0,emission
1,time
2,revenue
3,restrict
4,system
...,...
116,technic
117,diagnostic
118,contaminant
119,return


In [49]:
#각 데이터 프레임 중복처리
word_result = pre_result_word.drop_duplicates()

In [61]:
word_result.describe()

,Word
count,120
unique,120
top,emission
freq,1


In [71]:
refer_file_word = excel_file_load(dir_path, "참조모델데이터.xlsx", "Sheet2")
refer_file_word = refer_file_word.dropna().rename(columns={'Words3': 'Word'})
refer_file_word

,Word
0,active
1,actual
2,additional
3,air
4,alway
...,...
129,voltage
130,water
131,weight
132,width


In [72]:
merged_sheet = pd.concat([word_result, refer_file_word], ignore_index=True)


In [73]:
merged_sheet

,Word
0,emission
1,time
2,revenue
3,restrict
4,system
...,...
249,voltage
250,water
251,weight
252,width


In [75]:
merged_sheet = merged_sheet.drop_duplicates()
merged_sheet

,Word
0,emission
1,time
2,revenue
3,restrict
4,system
...,...
249,voltage
250,water
251,weight
252,width


In [78]:
# 결과 엑셀 시트에 분할 저장

result_file_path = os.path.join(dir_path,'참조모델 데이터 저장.xlsx')
with pd.ExcelWriter(result_file_path) as writer:
    merged_sheet.to_excel(writer, sheet_name='Sheet1', index=False)

In [83]:
refer_file = excel_file_load(dir_path, "참조모델데이터.xlsx", "한글 용어")
refer_file = refer_file.dropna()
refer_file

,용어
0,AAS 기반 자산 Lifecycle 정보
1,OEE 주요 지표
2,Summary&내용관리
3,가동 및 리드타임 단축
4,가동 및 최적 상태의 생산 운영 학습
...,...
535,효과 PQCD
536,효율 지표
537,후처리(중량측정/사상작업)
538,히터상태


In [86]:
from konlpy.tag import Hannanum

# Kkma 형태소 분석기 객체 생성
hannanum = Hannanum()

# 형태소 분석 결과를 저장할 함수 정의
def hannanum_morphs(text):
    return hannanum.morphs(text)

def hannanum_nouns(text):
    word_list = hannanum.nouns(text)
    filtered_word = [word for word in word_list if not re.search(r'\d|\W', word) and len(word) > 1]
    return filtered_word

def hannanum_pos_t(text):
    return hannanum.pos(text)

def hannanum_pos(text):
    word_list = []
    morph_list = hannanum.pos(text)
    for word, morp in morph_list:
        if "NN" in morp:
            word_list.append(word)
    return word_list

In [85]:
kr_word = pd.DataFrame()
kr_word['단어'] = refer_file['용어'].apply(hannanum_morphs)

kr_word

,단어
0,"[AAS, 기반, 자산, Lifecycle, 정보]"
1,"[OEE, 주요, 지표]"
2,[Summary&내용관리]
3,"[가동, 및, 리드타, 이, ㅁ, 단축]"
4,"[가동, 및, 최적, 상태, 의, 생산, 운영, 학습]"
...,...
535,"[효과, PQCD]"
536,"[효율, 지표]"
537,"[후처리, (, 중량측정, /, 사상작업, )]"
538,[히터상태]


In [92]:
kr_word = pd.DataFrame()
kr_word['단어'] = refer_file['용어'].apply(hannanum_nouns)

kr_word

,단어
0,"[기반, 자산, 정보]"
1,"[주요, 지표]"
2,[]
3,"[가동, 리드타, 단축]"
4,"[가동, 최적, 상태, 생산, 운영, 학습]"
...,...
535,[효과]
536,"[효율, 지표]"
537,"[후처리, 중량측정, 사상작업]"
538,[히터상태]


In [99]:
new_df = kr_word.explode('단어').reset_index(drop=True)
new_df = new_df.dropna()
new_df = new_df.drop_duplicates()
new_df

,단어
0,기반
1,자산
2,정보
3,주요
4,지표
...,...
1144,후처리
1145,중량측정
1146,사상작업
1147,히터상태


In [100]:
# 결과 엑셀 시트에 분할 저장

result_file_path = os.path.join(dir_path,'참조모델 데이터 저장.xlsx')
with pd.ExcelWriter(result_file_path) as writer:
    merged_sheet.to_excel(writer, sheet_name='영문 단어', index=False)
    new_df.to_excel(writer, sheet_name='한글 단어', index=False)